In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from scipy import stats
import datetime
# color-maps
import matplotlib.cm as cm

In [4]:
df = pd.read_csv("marketing_data.csv")

#### Step 1. Once data is imported, investigate variables like Dt_Customer and Income, etc., and check if they are imported correctly

In [5]:
df.sample(5)

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Response,Complain,Country
209,263,1945,PhD,Single,"$45,576.00",0,0,5/28/14,9,56,...,3,8,0,0,0,0,0,1,0,SP
393,9392,1971,Graduation,Single,"$26,954.00",1,0,5/8/14,17,4,...,2,7,0,0,0,0,0,0,0,SP
346,10507,1977,2n Cycle,Divorced,"$59,601.00",0,1,7/25/13,14,691,...,7,8,0,1,0,0,0,0,0,US
1974,4478,1979,Graduation,Married,"$63,777.00",1,1,3/24/13,87,457,...,6,8,0,0,0,0,0,0,0,IND
2028,4587,1944,Master,Widow,"$45,006.00",0,0,7/18/13,90,162,...,7,3,0,0,0,0,0,0,0,SP


In [6]:
df.shape

(2240, 28)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 28 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   2240 non-null   int64 
 1   Year_Birth           2240 non-null   int64 
 2   Education            2240 non-null   object
 3   Marital_Status       2240 non-null   object
 4    Income              2216 non-null   object
 5   Kidhome              2240 non-null   int64 
 6   Teenhome             2240 non-null   int64 
 7   Dt_Customer          2240 non-null   object
 8   Recency              2240 non-null   int64 
 9   MntWines             2240 non-null   int64 
 10  MntFruits            2240 non-null   int64 
 11  MntMeatProducts      2240 non-null   int64 
 12  MntFishProducts      2240 non-null   int64 
 13  MntSweetProducts     2240 non-null   int64 
 14  MntGoldProds         2240 non-null   int64 
 15  NumDealsPurchases    2240 non-null   int64 
 16  NumWeb

In [8]:
df.columns

Index(['ID', 'Year_Birth', 'Education', 'Marital_Status', ' Income ',
       'Kidhome', 'Teenhome', 'Dt_Customer', 'Recency', 'MntWines',
       'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',
       'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
       'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth',
       'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1',
       'AcceptedCmp2', 'Response', 'Complain', 'Country'],
      dtype='object')

It can be seen that **there are some columns** names in the dataset which **have leading or trailing spaces.** We will rename these columns.

In [9]:
df = df.rename(columns = lambda x : x.strip())

In [11]:
df = df.rename(columns = {'Income' : 'Income (In Dollars)'})

In [12]:
df.columns

Index(['ID', 'Year_Birth', 'Education', 'Marital_Status',
       'Income (In Dollars)', 'Kidhome', 'Teenhome', 'Dt_Customer', 'Recency',
       'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts',
       'MntSweetProducts', 'MntGoldProds', 'NumDealsPurchases',
       'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases',
       'NumWebVisitsMonth', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5',
       'AcceptedCmp1', 'AcceptedCmp2', 'Response', 'Complain', 'Country'],
      dtype='object')

The **columns have been renamed** and **leading** or **trailing spaces** have been removed.

#### Step 2. Income values for a few customers are missing. Perform missing value imputation. Assume that the customers with similar education and marital status make the same yearly income, on average. You may have to clean the data before performing this. For data cleaning, look into the categories of education and marital status. 

In [13]:
df.isnull().sum()

ID                      0
Year_Birth              0
Education               0
Marital_Status          0
Income (In Dollars)    24
Kidhome                 0
Teenhome                0
Dt_Customer             0
Recency                 0
MntWines                0
MntFruits               0
MntMeatProducts         0
MntFishProducts         0
MntSweetProducts        0
MntGoldProds            0
NumDealsPurchases       0
NumWebPurchases         0
NumCatalogPurchases     0
NumStorePurchases       0
NumWebVisitsMonth       0
AcceptedCmp3            0
AcceptedCmp4            0
AcceptedCmp5            0
AcceptedCmp1            0
AcceptedCmp2            0
Response                0
Complain                0
Country                 0
dtype: int64

It can be seen that there are **24 Missing values** in the column Income.

In [15]:
df[df.isnull().any(axis = 1)]

,ID,Year_Birth,Education,Marital_Status,Income (In Dollars),Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Response,Complain,Country
134,8996,1957,PhD,Married,NaN,2,1,11/19/12,4,230,...,8,9,0,0,0,0,0,0,0,GER
262,1994,1983,Graduation,Married,NaN,1,0,11/15/13,11,5,...,2,7,0,0,0,0,0,0,0,US
394,3769,1972,PhD,Together,NaN,1,0,3/2/14,17,25,...,3,7,0,0,0,0,0,0,0,AUS
449,5255,1986,Graduation,Single,NaN,1,0,2/20/13,19,5,...,0,1,0,0,0,0,0,0,0,AUS
525,8268,1961,PhD,Married,NaN,0,1,7/11/13,23,352,...,7,6,0,0,0,0,0,0,0,CA
590,10629,1973,2n Cycle,Married,NaN,1,0,9/14/12,25,25,...,3,8,0,0,0,0,0,0,0,GER
899,10475,1970,Master,Together,NaN,0,1,4/1/13,39,187,...,6,5,0,0,0,0,0,0,0,US
997,9235,1957,Graduation,Single,NaN,1,1,5/27/14,45,7,...,2,7,0,0,0,0,0,0,0,GER
1096,4345,1964,2n Cycle,Single,NaN,1,1,1/12/14,49,5,...,2,7,0,0,0,0,0,0,0,AUS
1185,7187,1969,Master,Together,NaN,1,1,5/18/13,52,375,...,4,3,0,0,0,0,0,0,0,AUS


In [19]:
df['Income (In Dollars)'] = df['Income (In Dollars)'].replace({"\$":'',",":''}, regex = True).astype(float)

In [20]:
df.describe(include='all')

,ID,Year_Birth,Education,Marital_Status,Income (In Dollars),Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Response,Complain,Country
count,2240.000000,2240.000000,2240,2240,2216.000000,2240.000000,2240.000000,2240,2240.000000,2240.000000,...,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240
unique,NaN,NaN,5,8,NaN,NaN,NaN,663,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8
top,NaN,NaN,Graduation,Married,NaN,NaN,NaN,8/31/12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SP
freq,NaN,NaN,1127,864,NaN,NaN,NaN,12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1095
mean,5592.159821,1968.805804,NaN,NaN,52247.251354,0.444196,0.506250,NaN,49.109375,303.935714,...,5.790179,5.316518,0.072768,0.074554,0.072768,0.064286,0.013393,0.149107,0.009375,NaN
std,3246.662198,11.984069,NaN,NaN,25173.076661,0.538398,0.544538,NaN,28.962453,336.597393,...,3.250958,2.426645,0.259813,0.262728,0.259813,0.245316,0.114976,0.356274,0.096391,NaN
min,0.000000,1893.000000,NaN,NaN,1730.000000,0.000000,0.000000,NaN,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
25%,2828.250000,1959.000000,NaN,NaN,35303.000000,0.000000,0.000000,NaN,24.000000,23.750000,...,3.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
50%,5458.500000,1970.000000,NaN,NaN,51381.500000,0.000000,0.000000,NaN,49.000000,173.500000,...,5.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
75%,8427.750000,1977.000000,NaN,NaN,68522.000000,1.000000,1.000000,NaN,74.000000,504.250000,...,8.000000,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN


In [23]:
df.groupby(['Education','Marital_Status'])['Income (In Dollars)'].mean()

Education   Marital_Status
2n Cycle    Divorced          49395.130435
            Married           46201.100000
            Single            53673.944444
            Together          44736.410714
            Widow             51392.200000
Basic       Divorced           9548.000000
            Married           21960.500000
            Single            18238.666667
            Together          21240.071429
            Widow             22123.000000
Graduation  Absurd            79244.000000
            Alone             34176.000000
            Divorced          54526.042017
            Married           50800.258741
            Single            51322.182927
            Together          55758.480702
            Widow             54976.657143
Master      Absurd            65487.000000
            Alone             61331.000000
            Divorced          50331.945946
            Married           53286.028986
            Single            53530.560000
            Together       

In [25]:
df['Income (In Dollars)'] = df.groupby(['Education','Marital_Status'])['Income (In Dollars)'].transform(lambda x : x.fillna(x.mean()))

In [26]:
df.isna().sum()

ID                     0
Year_Birth             0
Education              0
Marital_Status         0
Income (In Dollars)    0
Kidhome                0
Teenhome               0
Dt_Customer            0
Recency                0
MntWines               0
MntFruits              0
MntMeatProducts        0
MntFishProducts        0
MntSweetProducts       0
MntGoldProds           0
NumDealsPurchases      0
NumWebPurchases        0
NumCatalogPurchases    0
NumStorePurchases      0
NumWebVisitsMonth      0
AcceptedCmp3           0
AcceptedCmp4           0
AcceptedCmp5           0
AcceptedCmp1           0
AcceptedCmp2           0
Response               0
Complain               0
Country                0
dtype: int64

#### Step 3. Create variables to populate the total number of children, age, and total spending. 

In [28]:
df.columns

Index(['ID', 'Year_Birth', 'Education', 'Marital_Status',
       'Income (In Dollars)', 'Kidhome', 'Teenhome', 'Dt_Customer', 'Recency',
       'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts',
       'MntSweetProducts', 'MntGoldProds', 'NumDealsPurchases',
       'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases',
       'NumWebVisitsMonth', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5',
       'AcceptedCmp1', 'AcceptedCmp2', 'Response', 'Complain', 'Country'],
      dtype='object')

In [31]:
df['Total_Children'] = df['Kidhome'] + df['Teenhome']
df['Age'] = datetime.datetime.now().year - df['Year_Birth']
df['Total_Spending'] = df['MntWines'] + df['MntFruits'] + df['MntMeatProducts'] + df['MntFishProducts'] + df['MntSweetProducts'] + df['MntGoldProds']

Created 3 new columns: Total_Children, Age and Total_Spending.

In [32]:
df.sample(5)

,ID,Year_Birth,Education,Marital_Status,Income (In Dollars),Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Response,Complain,Country,Total_Children,Age,Total_Spending
479,9150,1963,PhD,Single,48918.0,1,1,4/12/14,21,52,...,0,0,0,0,0,0,SP,2,61,62
945,9360,1982,Graduation,Married,37040.0,0,0,8/8/12,41,86,...,0,0,0,0,0,0,US,0,42,316
116,11084,1976,Master,Together,65104.0,0,1,11/14/13,4,738,...,0,0,0,0,1,0,CA,1,48,1053
1154,5290,1964,PhD,Married,41551.0,1,1,8/14/13,51,220,...,0,0,0,0,0,0,CA,2,60,279
2071,5562,1974,Graduation,Married,60093.0,0,1,6/26/13,92,502,...,0,0,0,0,0,0,SP,1,50,735


#### Steps 4. Create box plots and histograms to understand the distributions and outliers. Perform outlier treatment.

In [34]:
Q1 = df['Income (In Dollars)'].quantile(0.25)
Q3 = df['Income (In Dollars)'].quantile(0.75)
IQR = Q3 - Q1

In [35]:
df = df[~((df['Income (In Dollars)'] > (Q3 + 1.5 * IQR)) | (df['Income (In Dollars)'] < (Q1 - 1.5 * IQR)))]

In [40]:
df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'])

<ipython-input-40-5ede84e7ffb2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'])


#### Step 5. Use ordinal encoding and one hot encoding according to different types of categorical variables

In [43]:
le = LabelEncoder()
df_origin = df.copy()

cat_cols = df.select_dtypes(include = ['object']).columns

In [44]:
cat_cols

Index(['Education', 'Marital_Status', 'Country'], dtype='object')

In [46]:
for col in cat_cols:
    df[col] = le.fit_transform(df[col])

<ipython-input-46-ac9cf5aadcdd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = le.fit_transform(df[col])


In [47]:
df.sample(5)

,ID,Year_Birth,Education,Marital_Status,Income (In Dollars),Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Response,Complain,Country,Total_Children,Age,Total_Spending
2232,7232,1973,2,6,42429.0,0,1,2014-02-11,99,55,...,0,0,0,0,0,0,6,1,51,67
1976,1118,1956,3,3,50965.0,0,1,2013-02-20,87,544,...,0,0,0,0,0,0,6,1,68,685
145,624,1984,3,4,18890.0,0,0,2012-11-10,5,6,...,0,0,0,0,1,0,5,0,40,35
892,8523,1968,2,3,19329.0,1,0,2013-12-14,39,24,...,0,0,0,0,0,0,1,1,56,60
877,5036,1984,4,4,42710.0,1,0,2012-11-28,38,252,...,0,0,0,0,0,0,6,1,40,406


#### Step 7.1: Older people are not as tech-savvy and probably prefer shopping in-store.

In [48]:
store_purchases = df[df['Age'] > 50]['NumStorePurchases']
web_purchases = df[df['Age'] > 50]['NumWebPurchases']

In [49]:
t_stat, p_value = stats.ttest_ind(store_purchases, web_purchases)

In [51]:
alpha = 0.05
if p_value <= alpha:
    print("People above 50 prefer shopping in-store")
else:
    print("People above 50 do shopping in-store and as well as online both equally.")

People above 50 prefer shopping in-store


#### Step 7.2: Customers with kids probably have less time to visit a store and would prefer to shop online.


In [52]:
store_purchases1 = df[df['Total_Children'] > 0]['NumStorePurchases']
web_purchases1 = df[df['Total_Children'] > 0]['NumWebPurchases']

In [54]:
t_stat, p_value = stats.ttest_ind(store_purchases1, web_purchases1)

In [55]:
alpha = 0.05
if p_value <= alpha:
    print("People with children prefer shopping in-store")
else:
    print("People with children do shopping in-store and as well as online equally.")

People with children prefer shopping in-store


In [56]:
mean_store_purchases1 = df[df['Total_Children'] > 0]['NumStorePurchases'].mean()
mean_web_purchases1 = df[df['Total_Children'] > 0]['NumWebPurchases'].mean()

In [57]:
if mean_store_purchases1 > mean_web_purchases1:
    print("People with children prefer shopping in-store")
else:
    print("People with children prefer shopping online.")

People with children prefer shopping in-store


Step 7.3: Other distribution channels may cannibalize sales at the store.

In [59]:
# compare overall NumStorePurchases, NumWebPurchases

#### Step 7.4: Does the US fare significantly better than the rest of the world in terms of total purchases?

In [60]:
df_origin['TotalPurchses'] = df['NumWebPurchases'] + df['NumStorePurchases'] + df['NumCatalogPurchases']
total_purchases_by_country = df_origin.groupby('Country')['TotalPurchses'].sum()

In [61]:
US_purchases = total_purchases_by_country['US']

In [62]:
rest_of_world_purchases = total_purchases_by_country.sum() - US_purchases

In [63]:
if US_purchases > rest_of_world_purchases:
    print("The US fares significantly better than the rest of the world in terms of total purchases.")
else:
    print("The US does not fare significantly better than the rest of the world in terms of total purchases.")

The US does not fare significantly better than the rest of the world in terms of total purchases.


In [64]:
t_stat, p_value = stats.ttest_ind(US_purchases, rest_of_world_purchases)

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3621: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [65]:
alpha = 0.05
if p_value <= alpha:
    print("The US fares significantly better than the rest of the world in terms of total purchases.")
else:
    print("The US does not fare significantly better than the rest of the world in terms of total purchases.")

The US does not fare significantly better than the rest of the world in terms of total purchases.


In [69]:
df_origin[df_origin['Complain'] == 1]['Education'].value_counts()

Graduation    14
2n Cycle       4
Master         2
PhD            1
Name: Education, dtype: int64